In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import split,col,filter,isnan,when
from pyspark.sql.types import StringType,IntegerType
import pandas as pd
import matplotlib.pyplot as plt
from petastorm.unischema import Unischema,UnischemaField
from petastorm.etl.dataset_metadata import materialize_dataset
from petastorm.codecs import ScalarCodec,NdarrayCodec
from petastorm.transform import TransformSpec
import numpy as np
import psutil
import os
import sys
#sys.path.append(os.path.join(os.getcwd(), ".."))
from operations.dataset_manager import get_dataset

## Get your dataset 

In [ ]:
name = "mit-bih-noise-stress-test-database-1.0.0"
dataset = get_dataset(name,fs=500,time_window=10)

In [ ]:
data_test = dataset["118e00"]

## Create data parquet file (the parquet scheme will be given)

### Checking your cpu core and usage

In [ ]:
##Check number of cpu core and memory
print(os.cpu_count())
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

### Ceating a Spark session

In [ ]:
spark = SparkSession.builder.appName('MyfirstSpark').config('spark.driver.memory', '2g').getOrCreate()

### Unischem use

In [ ]:
ECGSchemaPhysio = Unischema(
    "ECGSchema",
    [
        UnischemaField("noun_id", np.string_, (), ScalarCodec(StringType()), False),
        UnischemaField("signal_names", np.string_, (None,None), NdarrayCodec(), False),
        UnischemaField("signal", np.float32, (None, None, None), NdarrayCodec(), False),
        UnischemaField("number_of_window", np.int64, (None,), NdarrayCodec(), False),
        UnischemaField("score_classes", np.int64, (None,), NdarrayCodec(), False),
        UnischemaField("signal_length", np.int_, (), ScalarCodec(IntegerType()), False),
        UnischemaField("sampling_frequency", np.int_, (), ScalarCodec(IntegerType()), False),
    ],
)